In [9]:
import shared.config as config
import shared.const as const
import pymongo
import numpy as np
import pandas as pd
import json
import re
import ast

from pymongo import ReplaceOne
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

In [70]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]

stagingPlaatje = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
stagingDigiFoto = stagingDb[config.COLL_STAGING_DIGIFOTOS]
collection = analyseDb[config.COLL_ANALYSE_CLEAN]


In [71]:
soort = 'Artefact'
soort_lw = soort.lower()
key = 'key_foto1'
refkey = None

In [72]:
#df_soort = pd.DataFrame(list(cleancollection.find({"soort": "Artefact"}, projection={'key_foto1':1, 'ID':1}))) #find(soort_query, projection={key:1, 'ID':1})
#df_soort

soort_query = {'soort': soort}
if key == 'key':
    df_soort = pd.DataFrame(list(collection.find(soort_query, projection={key:1, 'ID':1})))

    key = 'key_' + soort_lw
    df_soort = df_soort.rename(columns={'_id': soort_lw+'UUID', 'index':soort_lw+'ID', 'key': key})
else:
    df_soort = pd.DataFrame(list(collection.find(soort_query, projection={key:1, 'ID':1})))
    df_soort = df_soort.rename(columns={'_id': soort_lw+'UUID', 'index':soort_lw+'ID'})

df_soort

,artefactUUID,ID,key_foto1
0,63cbfd7df061319b318728b4,0,NaN
1,63cbfd7df061319b318728b5,1,NaN
2,63cbfd7df061319b318728b6,2,NaN
3,63cbfd7df061319b318728b7,3,NaN
4,63cbfd7df061319b318728b8,4,NaN
...,...,...,...
23696,63cbfdba93835296b04309bf,23696,PDC112MHoutV96A701
23697,63cbfdb914a7c8d606075250,23697,PDC112MLeerV97A801
23698,63cbfdb914a7c8d606075251,23698,PDC112MLeerV97A802
23699,63cbfdb914a7c8d606075252,23699,PDC112MLeerV97A803


In [73]:
df_ref = pd.DataFrame(list(collection.find({key: {"$exists": True}}, projection={key:1, 'soort': 1})))
df_ref

,_id,soort,key_foto1
0,63cbfd9fbd333688f1c9e822,Bestand,PDB034MMetaalP10V1023A1
1,63cbfee6060e19fcd0497ca9,Bestand,PDB034MMetaalP10V1023A1
2,63cbfe73bd3d2afdf616cc4a,Bestand,PDB034MMetaalP10V1023A1
3,63cbff36bd333688f1c9ebaa,Bestand,PDB034MMetaalP10V1025A1
4,63cbfe8ac197e35871c99e0e,Bestand,PDB034MMetaalP10V1025A1
...,...,...,...
10314,63cbfdaee057bc465f835d0c,Artefact,PDC026MAardewerkP3V15A2
10315,63cbfdb84fa0d0a619d5ac9c,Artefact,PDC097MGlasP1V304A2004
10316,63cbfdad1fd67877c9eb3375,Artefact,PDC026MAardewerkP3V360A301
10317,63cbfdaee057bc465f836040,Artefact,PDC026MAardewerkP3V288A12


In [78]:
df_merge = pd.merge(df_ref, df_soort, how='left', on=key).rename(columns={'ID': (soort_lw if not refkey else refkey) + 'ID'})
df_merge[(df_merge.soort == 'Bestand') & (df_merge.artefactID.notnull())]

,_id,soort,key_foto1,artefactUUID,artefactID
16,63cbfddae5dd65b7bb662e5d,Bestand,PDB034MAardewerkP10V127A25,63cbfd7e80998abd616be99c,19502.0
25,63cbfe97c91764ca9abd89ea,Bestand,PDB034MAardewerkP10V1295A230,63cbfd8271b39e4d432fff45,22030.0
26,63cbfe63060e19fcd0497ba9,Bestand,PDB034MAardewerkP10V1295A230,63cbfd8271b39e4d432fff45,22030.0
27,63cbfd880cf0fd7136cdadbf,Bestand,PDB034MAardewerkP10V1295A230,63cbfd8271b39e4d432fff45,22030.0
28,63cbfe90bd333688f1c9eac5,Bestand,PDB034MAardewerkP10V1295A230,63cbfd8271b39e4d432fff45,22030.0
...,...,...,...,...,...
2001,63cbfd75c91764ca9abd86f3,Bestand,PDC097MHoutP1V92A701,63cbfdb7eb9ed91356483d72,18993.0
2002,63cbfe200cf0fd7136cdaf3f,Bestand,PDC097MHoutP1V92A701,63cbfdb7eb9ed91356483d72,18993.0
2068,63cbff13bd333688f1c9eb28,Bestand,PDC112MHoutV117A701,63cbfdba93835296b04309c1,23575.0
2069,63cbfe47bd333688f1c9e9ea,Bestand,PDC112MHoutV117A701,63cbfdba93835296b04309c1,23575.0


In [83]:
str = "/input/images/DC005_Anne Gesticht of Anna Convent/L Fotos/H (Objectfoto's)/Hout/DC05_P01_H0001_722_2.JPG"

'Hout/' in str

True

In [107]:
df = pd.DataFrame(list(collection.find({'soort': 'Bestand'})))
                                
df[df.rapporttype.notnull()]

,_id,fileName,imageID,imageMiddleID,imageThumbID,fileType,directory,mime_type,fototype,soort,...,rapportnr,titel,ciscode,periode_uitvoering,rob,kb,archief,type_onderzoek,key_rapport,rapporttype
5063,63cc00f5bd3d2afdf616cd63,DAN001.pdf,/rapportage/DAN001.pdf,/rapportage/DAN001-med.jpg,/rapportage/DAN001-sml.jpg,.pdf,/rapportage,application/pdf,R,Bestand,...,DAN001,Hendrik Casimirstraat - Professor Telderslaan ...,37974,11/09,0.0,0.0,0.0,IVO Boren,RDAN001,Archeologische Notitie
5064,63cc0025bd3d2afdf616cd13,DAN002.pdf,/rapportage/DAN002.pdf,/rapportage/DAN002-med.jpg,/rapportage/DAN002-sml.jpg,.pdf,/rapportage,application/pdf,R,Bestand,...,DAN002,Dompad 7 te De Lier (gemeente Westland). Arche...,38315,11/09,0.0,0.0,0.0,IVO Boren,RDAN002,Archeologische Notitie
5065,63cbfec1bd3d2afdf616cc97,DAN003.pdf,/rapportage/DAN003.pdf,/rapportage/DAN003-med.jpg,/rapportage/DAN003-sml.jpg,.pdf,/rapportage,application/pdf,R,Bestand,...,DAN003,Plangebied Hoogeland (2009) te Naaldwijk (geme...,"38257, 38258",11/09,0.0,0.0,0.0,IVO Boren,RDAN003,Archeologische Notitie
5066,63cc00b2bd3d2afdf616cd48,DAN004.pdf,/rapportage/DAN004.pdf,/rapportage/DAN004-med.jpg,/rapportage/DAN004-sml.jpg,.pdf,/rapportage,application/pdf,R,Bestand,...,DAN004,Noordlierweg te De Lier (gemeente Westland). B...,38313,11/09,0.0,0.0,0.0,IVO Boren,RDAN004,Archeologische Notitie
5067,63cc0020bd3d2afdf616cd11,DAN005.pdf,/rapportage/DAN005.pdf,/rapportage/DAN005-med.jpg,/rapportage/DAN005-sml.jpg,.pdf,/rapportage,application/pdf,R,Bestand,...,DAN005,Voormalige AZC-locatie (nabij Hoefweg 5A) te D...,39833,02/10,0.0,0.0,0.0,Bureauonderzoek,RDAN005,Archeologische Notitie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5494,63cc071c668c5d83b7f3fdc2,DAR144.pdf,/rapportage/DAR144.pdf,/rapportage/DAR144-med.jpg,/rapportage/DAR144-sml.jpg,.pdf,/rapportage,application/pdf,R,Bestand,...,DAR144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RDAR144,Archeologische Rapportage
5495,63cbfebdbd3d2afdf616cc95,DAR145 Vrijgestoven sporen in het Oude Duinzan...,/rapportage/DAR145 Vrijgestoven sporen in het ...,/rapportage/DAR145 Vrijgestoven sporen in het ...,/rapportage/DAR145 Vrijgestoven sporen in het ...,.pdf,/rapportage,application/pdf,R,Bestand,...,DAR145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RDAR145,Archeologische Rapportage
5496,63cc0627668c5d83b7f3fdac,DAR146_Archeologie_vd_Oude_Duinen_def.pdf,/rapportage/DAR146_Archeologie_vd_Oude_Duinen_...,/rapportage/DAR146_Archeologie_vd_Oude_Duinen_...,/rapportage/DAR146_Archeologie_vd_Oude_Duinen_...,.pdf,/rapportage,application/pdf,R,Bestand,...,DAR146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RDAR146,Archeologische Rapportage
5497,63cc07c7668c5d83b7f3fdea,"DAR96, Kaartbijlage 1 Verwachtingskaart.pdf","/rapportage/DAR96, Kaartbijlage 1 Verwachtings...","/rapportage/DAR96, Kaartbijlage 1 Verwachtings...","/rapportage/DAR96, Kaartbijlage 1 Verwachtings...",.pdf,/rapportage,application/pdf,R,Bestand,...,DAR96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RDAR96,Archeologische Rapportage


In [6]:
str = "DB34_G0001_03.jpg"
regex = r'^([a-zA-Z0-9]+)_([A-Z])([0-9]+).*\.[a-z]{3}$'
#regex = r'^([a-zA-Z0-9]+)_([ABCDEPT])(\d+)(_LZW)?$'

matchObj = re.match( regex, str, re.M|re.I)

matchObj.group(3)

'001'

In [24]:
from collections import namedtuple


def create_named_tuple(*values):
     return namedtuple('NamedTuple', values)(*values)


# First create tuple to later create Enum
bestand_tupels = create_named_tuple(
    const.BESTAND,
    const.BESTAND_OVERIGE,
    const.TEKENING,
    const.TEK_BOUWTEKENING, 
    const.TEK_VELDTEKENING, 
    const.TEK_OVERZICHTSTEKENING,
    const.TEK_OBJECTTEKENING,
    const.TEK_UITWERKINGSTEKENING,
    const.TEK_VELDTEKENING_PUBL,
    const.TEK_OBJECTTEKENING_PUBL,
    const.TEK_OVERIGE,
    const.FOTO,
    const.FOTO_SFEERFOTO,
    const.FOTO_OPGRAVINGSFOTO,
    const.FOTO_OBJECTFOTO,
    const.FOTO_OVERIGE,
    const.RAPPORTAGE,
    const.RAPP_ARCHEOLOGISCHE_RAPPORTAGE,
    const.RAPP_ARCHEOLOGISCHE_NOTITIE,
    const.RAPP_CONSERVERINGSRAPPORT,
    const.RAPP_OVERIGE_RAPPORTAGE
    )

[x for x in bestand_tupels if x.startswith('B')]

lst_all_bestanden = [b for b in bestand_tupels] 
lst_opgravingsfoto = [const.FOTO_OPGRAVINGSFOTO]
lst_objectfoto = [const.FOTO_OBJECTFOTO]
lst_veldtekening = [const.TEK_VELDTEKENING]
lst_archrapportage = [const.RAPP_ARCHEOLOGISCHE_RAPPORTAGE, const.RAPP_ARCHEOLOGISCHE_NOTITIE]
lst_overigerapportage = [const.RAPP_CONSERVERINGSRAPPORT, const.RAPP_OVERIGE_RAPPORTAGE]
lst_objecttekening = [const.TEK_OBJECTTEKENING, const.TEK_OBJECTTEKENING_PUBL]
lst_overigetekeningen = [const.TEKENING,
    const.TEK_BOUWTEKENING, 
    const.TEK_VELDTEKENING, 
    const.TEK_OVERZICHTSTEKENING,
    const.TEK_OBJECTTEKENING,
    const.TEK_UITWERKINGSTEKENING,
    const.TEK_VELDTEKENING_PUBL,
    const.TEK_OBJECTTEKENING_PUBL,
    const.TEK_OVERIGE]
lst_all_getoond = lst_opgravingsfoto + lst_objectfoto + lst_veldtekening + lst_archrapportage + lst_overigerapportage + lst_objecttekening + lst_overigetekeningen
lst_overige = list(set(lst_all_bestanden) - set(lst_all_getoond))
lst_overige

['Overige_afbeelding',
 'Foto',
 'Bestand_Overige',
 'Bestand',
 'Rapportage',
 'Sfeerfoto']